## 序列的修改,hashable和slice

* .[类定义](#类定义)
* .[序列属性](#支持序列属性)
* .[自定义属性](#自定义属性)
* .[hashable](#hashable)

### 类定义

In [8]:
import reprlib
import functools
import operator
from array import array 
from numbers import Integral

In [68]:


class Array:
    typecode = 'd'
    shortcut_names = 'xyzt'
    
    def __init__(self, component):
        self._components = array(self.typecode, component)
    
    def __len__(self):
        return len(self._components)
        
    def __getitem__(self, pos):
        """when indice is slice, the return type is also Array
        """
        cls = type(self)
        if isinstance(pos, slice):
            return cls(self._components[pos])
        elif instance(pos, Integral):
            return self._components[pos]
        else:
            raise TypeError(f'{cls.__name__} indices must be integers')

    def __getattr__(self, attr):
        cls = type(self)
        if len(attr) == 1:
            ind = cls.shortcut_names.find(attr)
            if 0 <= ind < len(self):
                return self._components[ind]
        msg = '{cls.__name__} has not attribute {attr}'
        raise AttributeError(msg.format(cls=cls, attr=attr))
    
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                err = 'readonly attribute {attr_name}'
            elif name.islower():
                err = "can't set attributes 'a' to 'z' in {cls_name}"
            else:
                err = ''
            
            if err:
                msg = err.format(attr_name=name, cls_name=cls.__name__)
                raise AttributeError(msg)
        super().__setattr__(name, value)
      
    # __eq__和__hash__使得hashable
    def __eq__(self, other):
        '''为什么&&不行,必须and呢?'''
        return len(self) == len(other) and all(a==b for a, b in zip(self, other))
    
    def __hash__(self):
        return functools.reduce(operator.xor, map(hash, self), 0)
        
    def __iter__(self):
        return iter(self._components)
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])) + bytes(array(self.typecode, self))
    
    def __repr__(self):
        components = reprlib.repr(self._components) 
        components = components[components.find('['):-1] 
        return 'Vector({})'.format(components)
    
    def __abs__(self):
        return math.sqrt(sum([x ** 2 for x in self]))
    
    def __bool__(self):
        return not abs(self) == 0

    @classmethod
    def frombytes(cls, bytes):
        typecode = chr(bytes[0])
        memv = memoryview(bytes[1:]).cast(typecode)
        return cls(*memv)

In [69]:
vector = Array(range(10))
vector

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

#### 序列属性
需要定义\_\_len\_\_和\_\_getitem\_\_,后者需要支持slice类

In [70]:
vector2 = vector[2:5]
vector2

Vector([2.0, 3.0, 4.0])

#### 自定义属性

需要定义\_\_getattr\_\_和\_\_setattr\_\_,后者用来防止行为不一致,比如语法中赋值了x,但是取值还是原值

In [71]:
vector2.z

4.0

In [72]:
vector.t

3.0

In [73]:
# vector2.t   # AttributeError: Array has not attribute t

AttributeError: Array has not attribute t

In [74]:
# vector2.x = 1   # AttributeError: readonly attribute x

AttributeError: readonly attribute x

### hashable
需要定义\_\_eq\_\_ 和\_\_hash\_\_

In [75]:
vector22 = Array(range(2, 5))
vector22

Vector([2.0, 3.0, 4.0])

In [76]:
vector2

Vector([2.0, 3.0, 4.0])

In [77]:
vector2 == vector22

True

In [82]:
hash(vector2)

5